In [1]:
import torch
import wandb
import numpy as np
from torch import nn
from torch import optim
from torch_geometric.nn import GCNConv, GATConv
from torch.utils.data import Dataset
from tqdm import tqdm
from sklearn.model_selection import KFold

from gfos.data.utils import load_layout
from gfos.utils.scheduler import CosineAnnealingWarmupRestarts
from gfos.metrics import metric_for_layout_collections
from gfos.loss import MultiElementRankLoss

device = "cuda" if torch.cuda.is_available() else "cpu"

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

## Configs

In [2]:
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"

configs = dict(
    conv_layer="GAT",
    num_epochs=100,
    learning_rate=5e-4,
    weight_decay=1e-6,
    min_lr=1e-7,
    warmup_ratio=0.05,
    max_configs=100,
    graph_hidden=[16, 32, 16, 48],
    graph_out=64,
    num_encoder=1,
    num_feedforward=512,
    nhead=1,
    loss_margin=0.1,
    loss_num_permutations=1000,
    accum_grad=4,
)

WANDB_PROJECT = "gfos"
WANDB_DIR = "../../logs/"
WANDB_RUN_NAME = "gcn_layout_xla"
TAGS = ["train", "layout", "xla"]


## Dataset

In [3]:
class LayoutDataset(Dataset):
    """Take all `c` configs as one batch. Using without dataloader."""

    def __init__(self, files: list[str], max_configs: int = -1, permute: bool = True):
        self.max_configs = max_configs
        self.files = files
        self.npzs = [np.load(file) for file in self.files]
        self.permute = permute
        self.num_records = [len(npz["config_runtime"]) for npz in self.npzs]

    def __len__(self):
        return len(self.npzs)

    def __getitem__(self, idx):
        # cum_records = np.cumsum(self.num_records)
        # npz_idx = np.searchsorted(cum_records, idx)
        # row = self.npzs[npz_idx]

        # cfg_idx = idx - cum_records[npz_idx - 1] if npz_idx > 0 else idx
        row = self.npzs[idx]
        max_configs = self.max_configs if self.max_configs > 0 else len(row["config_runtime"])

        target = torch.tensor(row["config_runtime"], dtype=torch.float)
        target = (target - target.mean()) / (target.std() + 1e-7)

        # Random sample `max_configs` configs from `c` configs
        if self.permute:
            config_indices = torch.randperm(target.size(0))[: max_configs]
        else:
            config_indices = torch.arange(max_configs)
        target = target[config_indices]

        node_feat = torch.tensor(row["node_feat"], dtype=torch.float)
        node_opcode = torch.tensor(row["node_opcode"], dtype=torch.long)
        edge_index = torch.tensor(
            np.swapaxes(row["edge_index"], 0, 1), dtype=torch.long
        )

        node_config_feat = torch.tensor(
            row["node_config_feat"], dtype=torch.float
        )[config_indices]

        node_config_ids = torch.tensor(
            row["node_config_ids"], dtype=torch.long
        )

        return (
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
            target,
        )

## Model

In [4]:
# def transform_node_positional_embeddings(
#     embeddings_output: torch.Tensor,
#     node_config_ids: torch.Tensor,
#     num_nodes: int,
# ) -> torch.Tensor:
#     num_configs, _, dim = embeddings_output.shape
#     idxs = node_config_ids.unsqueeze(0).repeat(num_configs, 1) # [c, nc]
#     zeros = torch.zeros(
#         num_configs,
#         num_nodes,
#         dim,
#         device=embeddings_output.device,
#         dtype=embeddings_output.dtype,
#     )
#     idxs = idxs.unsqueeze(-1).repeat(1, 1, dim) # [c, nc, dim]
#     zeros.scatter_reduce_(1, idxs, embeddings_output, reduce="sum")
#     return zeros

In [5]:
class LayoutModel(torch.nn.Module):
    def __init__(
        self,
        hidden_channels,
        graph_out,
        num_encoder=1,
        num_feedforward=256,
        nhead=1,
    ):
        super().__init__()

        op_embedding_dim = 32
        config_dim = 64
        merged_node_dim = graph_out + config_dim

        self.embedding = torch.nn.Embedding(
            120,
            op_embedding_dim,
        )
        assert len(hidden_channels) > 0
        in_channels = op_embedding_dim + 140
        self.convs = torch.nn.ModuleList()
        last_dim = hidden_channels[0]

        self.convs.append(GATConv(in_channels, hidden_channels[0]))
        for i in range(len(hidden_channels) - 1):
            self.convs += [
                GATConv(hidden_channels[i], hidden_channels[i + 1]),
            ]
            last_dim = hidden_channels[i + 1]
        self.convs.append(GATConv(last_dim, graph_out))

        # Transformer encoder to merge configs features with graph features
        layer = nn.TransformerEncoderLayer(
            d_model=merged_node_dim,
            dim_feedforward=num_feedforward,
            nhead=nhead,
            batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(layer, num_layers=num_encoder)

        self.layernorm = nn.LayerNorm(merged_node_dim)

        self.config_prj = nn.Sequential(
            nn.Linear(18, config_dim),
            nn.LayerNorm(config_dim),
            nn.ReLU(),
        )

        # Define a sequential dense neural network
        self.dense = torch.nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(merged_node_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(
        self,
        node_feat: torch.Tensor,
        node_opcode: torch.Tensor,
        edge_index: torch.Tensor,
        node_config_feat: torch.Tensor,
        node_config_ids: torch.Tensor,
    ) -> torch.Tensor:
        # Get graph features
        c = node_config_feat.size(0)

        x = torch.cat([node_feat, self.embedding(node_opcode)], dim=1)

        # Through convolutional layers
        for conv in self.convs:
            x = conv(x, edge_index).relu()

        x = x[node_config_ids]  # (N, 64) -> (NC, 64)
        
        node_config_feat = self.config_prj(node_config_feat) # (C, NC, 18) -> (C, NC, config_dim)
        x = torch.cat([x.repeat((c, 1, 1)), node_config_feat], dim=-1)  # (C, NC, config_dim + graph_out)
        x = nn.functional.normalize(x, dim=-1)

        x = self.encoder(x)[:, -1, :]  # (C, NC, config_dim + graph_out) -> (C, config_dim + graph_out)
        x = self.dense(x).flatten()

        return x

## Valid

In [6]:
layout_random = load_layout(
    LAYOUT_DIR,
    model_type="xla",
    compile_type="random",
)
val_dataset = LayoutDataset(files=layout_random["test"], permute=False)
_INFERENCE_CONFIGS_BATCH_SIZE = 100

num_epochs = configs["num_epochs"]
learning_rate = configs["learning_rate"]
weight_decay = configs["weight_decay"]
min_lr = configs["min_lr"]
warmup_ratio = configs["warmup_ratio"]
max_configs = configs["max_configs"]
graph_hidden = configs["graph_hidden"]
graph_out = configs["graph_out"]
num_encoder = configs["num_encoder"]
num_feedforward = configs["num_feedforward"]
nhead = configs["nhead"]
margin = configs["loss_margin"]
number_permutations = configs["loss_num_permutations"]
accum_grad = configs["accum_grad"]

model = LayoutModel(
    hidden_channels=graph_hidden,
    graph_out=graph_out,
    num_encoder=num_encoder,
    num_feedforward=num_feedforward,
    nhead=nhead,
).to(device)
static_dict = torch.load("../../logs/gcn_layout_xla/20230921_182345/52_0.0123.pth", map_location=device)
model.load_state_dict(static_dict)
model = model.eval()

In [11]:
layout_xla_scores = []
pred_idx = []


with torch.no_grad():
    for data in tqdm(val_dataset):
        (
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
            target,
        ) = data
        (
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
        ) = (
            node_feat.to(device),
            node_opcode.to(device),
            edge_index.to(device),
            node_config_feat.to(device),
            node_config_ids.to(device),
        )
        target = target.numpy()
        num_configs = target.shape[-1]
        outs = []

        for i in range(0, num_configs, _INFERENCE_CONFIGS_BATCH_SIZE):
            end_i = min(i + _INFERENCE_CONFIGS_BATCH_SIZE, num_configs)
            out: torch.Tensor = model(
                node_feat,
                node_opcode,
                edge_index,
                node_config_feat[i: end_i],
                node_config_ids,
            )
            outs.append(out.detach().cpu())
            pred_idx.append(np.argsort(out.detach().cpu().numpy()))

        score = metric_for_layout_collections(np.argsort(torch.concat(outs).numpy()), np.argsort(target))
        layout_xla_scores.append(score)


100%|██████████| 8/8 [00:02<00:00,  2.86it/s]


In [12]:
np.mean(layout_xla_scores)

-0.001862294362294362

In [ ]:
for file in tqdm(layout_random["test"]):
    npz = np.load(file)
    